<a href="https://colab.research.google.com/github/abduljabbar2017/project-Tensorflow/blob/main/TensorFlow_NN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Classifying Images of handwritten digits**

This guide trains a neural network model to classify images of handwritten digits.

This guide uses tf.keras, a high-level API to build and train models in TensorFlow.

In [ ]:
#Set up GPU for training
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!nvidia-smi

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Helper libraries
import os
import numpy as np
import matplotlib.pyplot as plt

# **Import the Fashion MNIST dataset**

This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images.

In [ ]:
mnist=keras.datasets.mnist

In [ ]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [ ]:
#Likewise, there are 60,000 labels in the training set:
# And the test set contains 10,000 images labels:
print("Number of training examples: {}".format(len(x_train)))
print("Number of test examples:     {}".format(len(x_test)))

In [ ]:
print(x_train.shape)
print(x_test.shape)

**Plotting the MNIST dataset using matplotlib**

In [ ]:
from matplotlib import pyplot
for i in range(9):
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(x_train[i], cmap=pyplot.get_cmap('gray'))
pyplot.show()

In [ ]:
x_train[1]

## Preprocess the data
The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255.

Scale these values to a range of 0 to 1 before feeding them to the neural network model. To do so, divide the values by 255. It's important that the training set and the testing set be preprocessed in the same way:

In [ ]:
# convert from integers to floats
train_norm = x_train.astype('float32')
test_norm = x_test.astype('float32')
# normalize to range 0-1
train_norm = train_norm / 255.0
test_norm = test_norm / 255.0

In [ ]:
test_norm[1]

# **Build the model**
Building the neural network requires configuring the layers of the model, then compiling the model.
## Setup the layers
The basic building block of a neural network is the layer. A layer extracts a representation from the data fed into it. Hopefully, a series of connected layers results in a representation that is meaningful for the problem at hand.

Much of deep learning consists of chaining together simple layers. Most layers, like tf.keras.layers.Dense, have internal parameters which are adjusted ("learned") during training.

In [ ]:
model=keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28,1)))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))
model.summary()

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's *compile* step:


* *Loss function* — An algorithm for measuring how far the model's outputs are from the desired output. The goal of training is this measures loss.
* *Optimizer* —An algorithm for adjusting the inner parameters of the model in order to minimize loss.
* *Metrics* —Used to monitor the training and testing steps. The following example uses *accuracy*, the fraction of the images that are correctly classified.

In [ ]:
loss=keras.losses.SparseCategoricalCrossentropy()
optim=keras.optimizers.Adam(learning_rate=0.001)
metrics=['acc']

In [ ]:
model.compile(loss=loss,optimizer=optim,metrics=metrics)

In [ ]:
batch_size=64
epochs = 5

In [ ]:
model.fit(train_norm,y_train,batch_size=batch_size,epochs=epochs,shuffle=True,verbose=2)# verbose = 0 mean no output, 1 mean progress bar and 2 mean logging

## Evaluate accuracy
Next, compare how the model performs on the test dataset. Use all examples we have in the test dataset to assess accuracy.

In [ ]:
test_loss, test_accuracy = model.evaluate(test_norm,y_test,batch_size=batch_size,verbose=2)
print('Accuracy on test dataset:', test_accuracy)

In [ ]:
model.save('digit_model.h5')

## Make predictions and explore

With the model trained, we can use it to make predictions about some images.

In [ ]:
# Select test image (replace 0 with desired index)
test_image = x_test[1]

# Preprocess the test image (normalize pixel values here)
test_image = test_image.astype('float32') / 255.0
test_image = np.expand_dims(test_image, axis=0)  # Reshape for prediction

# Load the trained model
model = tf.keras.models.load_model('digit_model.h5')

# Make prediction
predictions = model.predict(test_image)

# Get the predicted digit class (index of max probability)
predicted_class = np.argmax(predictions[0])

# Print the prediction result
print("Predicted digit:", predicted_class)

# Optionally, plot the test image
plt.imshow(test_image.reshape(28, 28), cmap='gray')
plt.title("Predicted Digit: " + str(predicted_class))
plt.show()